In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os 
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split 
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import cv2
import matplotlib.pyplot as plt
from IPython.display import Image
%matplotlib inline

import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
import numpy as np
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_auc_score

from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import ConfusionMatrixDisplay


from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import train_test_split

In [ ]:
!unzip '/content/drive/MyDrive/Pediatric Chest X-ray Pneumonia_CBAM.zip'

directory='/content/Pediatric Chest X-ray Pneumonia'
train_directory=os.path.join(directory,'train')
test_directory=os.path.join(directory,'test')
val_directory=os.path.join(directory,'val')

train_datagen = ImageDataGenerator(rescale=1/255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True)

validation_datagen = ImageDataGenerator(rescale=1/255)

test_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 32 using train_datagen generator
train_generator = train_datagen.flow_from_directory( train_directory, 
        classes = ['NORMAL', 'PNEUMONIA'],
        target_size=(224, 224),  
        batch_size=32,
        class_mode='binary')

# Flow validation images in batches of 32 using valid_datagen generator
validation_generator = validation_datagen.flow_from_directory( val_directory,
        classes = ['NORMAL', 'PNEUMONIA'],
        target_size=(224, 224), 
        batch_size=32,
        class_mode='binary',
        shuffle=False)

test_generator = test_datagen.flow_from_directory(test_directory,
        classes = ['NORMAL', 'PNEUMONIA'],
        target_size=(224, 224), 
        batch_size=32,
        class_mode='binary',
        shuffle=False)

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VAL=validation_generator.n//validation_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
print("STEP_SIZE_TRAIN: ",STEP_SIZE_TRAIN)
print("STEP_SIZE_TEST: ",STEP_SIZE_TEST)

In [ ]:
def cbam_block(cbam_feature, ratio=16):
    cbam_feature = channel_attention(cbam_feature, ratio)
    cbam_feature = spatial_attention(cbam_feature)
    return cbam_feature

def channel_attention(input_feature, ratio=16):

    if(tf.keras.backend.image_data_format()=="channels_first"):
      channel_axis=1
    else:
        channel_axis=-1

    filters = input_feature.shape[channel_axis]
    se_shape = (1, 1, filters)

    se_GP = tf.keras.layers.GlobalAveragePooling2D()(input_feature)
    se_xxxxx = tf.keras.layers.Reshape(se_shape)(se_GP)
    
    se = Dense(filters // ratio, activation='relu', use_bias=False)(se_xxxxx)
    se = Dense(filters, activation='sigmoid',  use_bias=False)(se_xxxxx)
    
    print(se.shape)
    print(se_xxxxx.shape)
    
    x = tf.keras.layers.Multiply()([se_xxxxx, se])

    

    return x

def spatial_attention(input_feature):
    
    kernel_size = 7
    channel = input_feature.shape[-1]
    cbam_feature = input_feature

    avg_pool = tf.keras.layers.Lambda(lambda x: tf.keras.backend.mean(x, axis=3, keepdims=True))(cbam_feature)
    max_pool = tf.keras.layers.Lambda(lambda x: tf.keras.backend.max(x, axis=3, keepdims=True))(cbam_feature)

    concat =  tf.keras.layers.Concatenate(axis=3)([avg_pool, max_pool]) 
    cbam_feature = tf.keras.layers.Conv2D(filters = 1,
                    kernel_size=kernel_size,
                    strides=1,
                    padding='same',
                    activation='sigmoid',
                    kernel_initializer='he_normal',
                    use_bias=False)(concat)
    
    return tf.keras.layers.multiply([input_feature, cbam_feature])


In [ ]:
model = tf.keras.applications.resnet_v2.ResNet50V2(include_top = False, weights="imagenet", input_shape=(224,224,3))

cbam_output = cbam_block(model.output)
global_average = tf.keras.layers.GlobalAveragePooling2D()(cbam_output)
dens= tf.keras.layers.Dense(512,activation='relu')(global_average)
dp= tf.keras.layers.Dropout(0.2)(dens)
final_classification_layers = tf.keras.layers.Dense(1,activation='sigmoid')(dp)
post_cbam_model = Model(model.input, final_classification_layers)

post_cbam_model.summary()

In [ ]:
cp = tf.keras.callbacks.ModelCheckpoint(filepath='POST_CBAM_Resnet50v2_comratio_16{epoch:02d}.h5',verbose=1, save_freq= int(2 * STEP_SIZE_TRAIN))

post_cbam_model.compile(optimizer=Adam(0.00001), loss = 'binary_crossentropy', metrics = 'accuracy')
history = post_cbam_model.fit(train_generator,
steps_per_epoch=STEP_SIZE_TRAIN,
class_weight= {0:1.939,1:0.674},
epochs=30,
verbose=1,
validation_data = validation_generator,
validation_steps=STEP_SIZE_VAL,
callbacks=[cp])

In [ ]:
preds = post_cbam_model.predict(test_generator,verbose=1)
final_preds=[]
for i in preds:
    if(i >= 0.5):
        final_preds.append(1)
    else:
        final_preds.append(0)
accuracy = accuracy_score(list(test_generator.classes), final_preds)
print('Accuracy:', "{:.2f}".format(accuracy*100))
precision = precision_score(list(test_generator.classes), final_preds)
print('Precision:', "{:.2f}".format(precision*100))
recall = recall_score(list(test_generator.classes), final_preds)
print('Recall:', "{:.2f}".format(recall*100))
f1 = f1_score(list(test_generator.classes), final_preds)
print('F1_score:', "{:.2f}".format(f1*100))
fpr, tpr, _ = roc_curve(list(test_generator.classes), final_preds)
roc_auc = auc(fpr, tpr)
print('ROC AUC:', "{:.2f}".format(roc_auc*100))
pr, tpr, _ = roc_curve(list(test_generator.classes), final_preds)
roc_auc = auc(fpr, tpr)
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

In [ ]:
post_cbam_model.save_weights("/content/drive/MyDrive/POST_CBAM_Resnet50v2_comratio_16_secbam.h5")

In [ ]:
plt.plot(history.history['accuracy'],marker = 'v')
plt.plot(history.history['val_accuracy'],marker = '*')
plt.title('ResNet50 V2(cbam ratio=16) Model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.ylim([0, 1])
plt.yticks(np.arange(0, 1, 0.1))
plt.show()

In [ ]:
plt.plot(history.history['loss'],marker = 'v')
plt.plot(history.history['val_loss'],marker = '*')
plt.title('ResNet50 V2(cbam ratio=16) Model loss')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.ylim([0, 1])
plt.yticks(np.arange(0, 1, 0.1))
plt.show()

In [ ]:
vgg19_model = tf.keras.models.load_model('/content/POST_CBAM_vgg19_compration64.h5')
resnet152v2_model = tf.keras.models.load_model('/content/POST_CBAM_resnet152v2_compratio128_ND.h5')

In [ ]:
vgg19 = [95.96,97.44,96.69,95.3]
resnet152v2 = [96.01,98.72,97.35,95.94]

vgg19 = np.array(vgg19) * 0.01
resnet152v2 = np.array(resnet152v2) * 0.01

In [ ]:
vgg19_tanh = sum(tf.keras.activations.tanh(tf.constant(vgg19, dtype = tf.float32)).numpy())
resnet152v2_tanh = sum(tf.keras.activations.tanh(tf.constant(resnet152v2, dtype = tf.float32)).numpy())
print(vgg19_tanh)
print(resnet152v2_tanh)

In [ ]:
vgg19_preds = vgg19_model.predict(test_generator,verbose=1)
resnet152v2_preds = resnet152v2_model.predict(test_generator,verbose=1)

In [ ]:
ens = []
for i in range(len(vgg19_preds)):
    num = vgg19_tanh*vgg19_preds[i] + resnet152v2_tanh*resnet152v2_preds[i]
    denom = vgg19_tanh + resnet152v2_tanh
    res = num/denom
    ens.append(res)

In [ ]:
final_preds=[]
for i in ens:
    if(i >= 0.5):
        final_preds.append(1)
    else:
        final_preds.append(0)
accuracy = accuracy_score(list(test_generator.classes), final_preds)
print('Accuracy:', "{:.2f}".format(accuracy*100))
precision = precision_score(list(test_generator.classes), final_preds)
print('Precision:', "{:.2f}".format(precision*100))
recall = recall_score(list(test_generator.classes), final_preds)
print('Recall:', "{:.2f}".format(recall*100))
f1 = f1_score(list(test_generator.classes), final_preds)
print('F1_score:', "{:.2f}".format(f1*100))
fpr, tpr, _ = roc_curve(list(test_generator.classes), final_preds)
roc_auc = auc(fpr, tpr)
print('ROC AUC:', "{:.2f}".format(roc_auc*100))
pr, tpr, _ = roc_curve(list(test_generator.classes), final_preds)
roc_auc = auc(fpr, tpr)
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()